In [30]:
import numpy as np
import matplotlib.pyplot as plt
from files.functions.results_extractors import *
import seaborn as sns
sns.set()
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": ["Palatino"]})

In [31]:
def isfloat(s):
    try:
        float(s)
        return True
    except:
        return False

In [32]:
def get_thresholds_aucs(dic):
    thresholds, aucs = [], []
    for threshold, dict3 in dic.items():
        if isfloat(threshold):
            thresholds.append(float(threshold))
            aucs.append(dict3['roc_auc'])

    idxs = np.argsort(thresholds)
    thresholds = np.array(thresholds)[idxs]
    aucs = np.array(aucs)[idxs]
    return np.dstack((thresholds, aucs))[0]

In [33]:
base_path = r"/home/catonz/Neural-PreferenceIsolation/server/results/ae_24142/"

def make_aucs_thresholds_plots(base_path, to_plot=False, to_save=False):
    dirs = [p for p in os.listdir(base_path) if "test" in p]
    params_results = np.array([[read_dict_json(path=joinpath(base_path, d, "parameters.json")), extract_results_models(joinpath(base_path, d))] for d in dirs])

    # sort based on mss values. From smaller to bigger
    params_results = params_results[np.argsort(list(map(lambda x: x["mss"], params_results[:,0])))]

    std_multiples = [1, 4, 7, 10, 25, 50, 100]
    sigma = r"$\sigma$"
    psi = r"$\psi$"
    for model, dict1 in params_results[0, 1].items():
        for ds_name, dict2 in dict1.items():
            fig = plt.figure(dpi=100, figsize=(10, 7))
            fig.suptitle(f"Dataset: {ds_name.replace('_', '-')}. {sigma} = {params_results[0, 0]['datasets_std'][ds_name]}", fontsize=20, y=0.93)

            all_aucs = np.array([])
            for params, results in params_results:
                res = get_thresholds_aucs(results[model][ds_name])
                thresholds, aucs = res[:,0], res[:,1]
                plt.plot(thresholds, aucs, marker=".", label=f"{psi} = {params['mss']}", )
                all_aucs = np.append(all_aucs, aucs)
            
            min_y = np.min(all_aucs)
            max_y = np.max(all_aucs)

            plt.xlabel("Threshold")
            plt.ylabel("AUC")
            plt.gca().yaxis.set_label_position("right")

            plt.ylim(min_y - 0.01, max_y + 0.01)
            plt.xticks(thresholds, labels=[f"{i}*{sigma}" for i in std_multiples], rotation=45, fontsize=10)

            plt.yticks(np.linspace(min_y, max_y, 9))
            plt.legend(loc="lower right");

            if to_plot:
                plt.show()
            if to_save:
                save_path = joinpath(base_path, "ds_aucs_plots")
                os.makedirs(save_path, exist_ok=True)
                fig.savefig(joinpath(save_path, f"{ds_name}.svg"), bbox_inches="tight")
            plt.close()

In [37]:
base_path = r"/home/catonz/Neural-PreferenceIsolation/server/results/ae_24142/"
best_ae_24142 = make_aucs_thresholds_plots(base_path)

In [35]:
base_path = r"/home/catonz/Neural-PreferenceIsolation/server/results/som/pif_benchmark"
som_55 = make_aucs_thresholds_plots(base_path)

In [36]:
 base_path = r"/home/catonz/Neural-PreferenceIsolation/server/results/ae_2841482/"
 best_ae_2841482 = make_aucs_thresholds_plots(base_path)